In [1]:
import json
import pandas as pd
import numpy as np
from numpy import array

import torch
import torch.nn as nn
import torch.optim as optim

import os
import warnings
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV



In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Loading RF Data Structures

In [3]:
# Load the data
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data")
df_rf_15 = pd.read_csv('df_rf_15.csv')
df_rf_20 = pd.read_csv('df_rf_20.csv')
df_rf_25 = pd.read_csv('df_rf_25.csv')
df_rf_30 = pd.read_csv('df_rf_30.csv')
df_rf_35 = pd.read_csv('df_rf_35.csv')
df_rf_40 = pd.read_csv('df_rf_40.csv')
df_rf_45 = pd.read_csv('df_rf_45.csv')
df2_allhero = pd.read_csv('df2_allhero.csv')

# Load the model
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/")
%run LSTM_Model_Classes_dynamic.ipynb

os.chdir("/content/drive/MyDrive/Colab Notebooks/698/")
%run Useful_Functions.ipynb



In [4]:
df_rf_15[['rad_pos1_last_gold','rad_pos2_last_gold','rad_pos3_last_gold','rad_pos4_last_gold', 'rad_pos5_last_gold',
          'dire_pos1_last_gold','dire_pos2_last_gold','dire_pos3_last_gold','dire_pos4_last_gold', 'dire_pos5_last_gold',
          'pos1_diff','pos2_diff','pos3_diff','pos4_diff','pos5_diff','team_diff']]

,match_id,rad_pos1,rad_pos2,rad_pos3,rad_pos4,rad_pos5,rad_pos1_gold,rad_pos2_gold,rad_pos3_gold,rad_pos4_gold,...,dire_pos4_last_gold,dire_pos5_last_gold,pos1_diff,pos2_diff,pos3_diff,pos4_diff,pos5_diff,team_diff,minute,winner
0,7517376613,25,8,84,26,5,"[0, 357, 715, 1130, 1578, 1968, 2261, 2861, 33...","[0, 246, 591, 895, 1255, 1499, 1823, 2117, 241...","[0, 247, 580, 826, 1235, 1481, 1876, 2279, 265...","[0, 170, 260, 350, 440, 827, 1051, 1141, 1278,...",...,2816,2692,445,-1062,-315,-207,-455,-1594,15,dire
1,7517420109,80,70,23,63,100,"[0, 245, 482, 901, 1226, 1720, 2196, 2613, 312...","[0, 243, 397, 859, 1111, 1554, 2117, 2596, 304...","[0, 244, 633, 916, 1375, 1957, 2416, 2835, 341...","[0, 216, 331, 513, 649, 779, 914, 1004, 1219, ...",...,2866,2798,-3138,-41,1095,-68,-170,-2322,15,radiant
2,7517437309,10,104,19,45,64,"[0, 197, 439, 738, 1066, 1445, 2121, 2750, 337...","[0, 290, 549, 944, 1402, 1785, 2309, 2676, 315...","[0, 256, 589, 718, 1033, 1536, 1798, 2094, 263...","[0, 173, 263, 742, 832, 969, 1207, 1400, 1688,...",...,2700,3026,254,279,941,821,-766,1529,15,dire
3,7517480603,1,33,11,100,30,"[0, 242, 587, 827, 1334, 1784, 2217, 3051, 345...","[0, 244, 898, 1268, 1718, 2173, 2632, 3024, 34...","[0, 289, 454, 837, 1306, 1601, 1904, 2157, 269...","[0, 169, 325, 415, 535, 670, 805, 895, 985, 13...",...,3095,2296,1572,1184,1554,-724,475,4061,15,radiant
4,7517522012,69,10,76,83,86,"[155, 492, 863, 1265, 1666, 2338, 2971, 3440, ...","[0, 261, 673, 1043, 1391, 1893, 2311, 2839, 31...","[155, 562, 775, 1065, 1468, 1948, 2310, 2798, ...","[89, 414, 504, 689, 861, 999, 1138, 1488, 1654...",...,3808,2856,1493,863,273,605,527,3761,15,dire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,7633793219,48,35,129,85,86,"[0, 246, 491, 893, 1141, 1572, 1908, 2550, 297...","[0, 200, 533, 935, 1341, 1787, 2351, 3132, 347...","[0, 216, 419, 809, 979, 1590, 1719, 1979, 2285...","[0, 129, 219, 339, 429, 977, 1280, 1939, 2146,...",...,3276,2775,-1161,1099,-515,1183,44,650,15,dire
991,7633814106,18,137,98,5,27,"[61, 338, 802, 1218, 1763, 2129, 2297, 2693, 3...","[61, 342, 727, 1127, 1374, 1816, 2144, 2491, 2...","[61, 343, 616, 846, 1212, 1536, 1790, 2148, 25...","[329, 538, 752, 926, 1117, 1252, 1342, 1480, 1...",...,3196,3610,2,-2171,126,345,-995,-2693,15,radiant
992,7633956117,70,96,23,31,123,"[0, 362, 569, 1042, 1264, 1949, 2268, 2527, 28...","[0, 253, 489, 735, 1118, 1377, 1549, 1932, 218...","[0, 169, 371, 622, 855, 1030, 1268, 1489, 1844...","[0, 233, 337, 574, 984, 1500, 1639, 1729, 1819...",...,3735,3792,-1872,-3323,-2267,-235,-1764,-9461,15,dire
993,7634024715,94,17,29,27,66,"[0, 243, 539, 889, 1138, 1579, 2192, 2523, 293...","[0, 375, 745, 909, 1238, 1583, 1824, 2368, 290...","[0, 295, 579, 784, 1149, 1470, 1747, 1949, 228...","[0, 170, 260, 350, 489, 628, 752, 842, 978, 11...",...,3175,3841,-1051,-612,-903,-506,-859,-3931,15,radiant


In [ ]:
def iterate_ids(id_array):
  # Takes in a numpy array, and feeds the values one by one into the embedding layer
  # Returns the embedding representation that can replace the column
  hero_embeddings_list = []
  for hero_id in id_array:
      embedding = RMSE_embedding(hero_id).detach().numpy()
      hero_embeddings_list.append(embedding)

  #print(hero_embeddings_list)

  hero_embeddings_tensor = torch.tensor(hero_embeddings_list)
  return hero_embeddings_tensor

In [ ]:
# split the data?
def drop_columns_for_predicted_df(df):
  cols_to_drop = df.filter(regex=r'\d_gold$|\d_last_gold$').columns #removing tensor columns
  df = df.drop(columns= cols_to_drop)
  return df

def drop_columns_for_no_embed(df):
  cols_to_drop = df.filter(regex=r'\d_gold$|\d_last_gold$|rad_pos\d$|dire_pos\d$|minute|match_id$').columns
  df = df.drop(columns= cols_to_drop)
  return df


def get_hero_embedding_and_concat_features(df):
  # Extract the hero_ids for each position from df

  hero_columns = ['rad_pos1', 'rad_pos2', 'rad_pos3', 'rad_pos4', 'rad_pos5',
                'dire_pos1', 'dire_pos2', 'dire_pos3', 'dire_pos4', 'dire_pos5']

  hero_embeddings = {col: iterate_ids(df[col].values) for col in hero_columns}

  # Create new columns for each hero position's embedded representations
  embedded_columns = {}
  for col in hero_columns:
      embeddings = hero_embeddings[col].detach().numpy()
      embeddings = embeddings.reshape(embeddings.shape[0],-1)
      #print(embeddings.shape)
      num_rows, num_features = embeddings.shape
      for i in range(num_features):
        embedded_columns[f"{col}_embed_{i+1}"] = embeddings[:, i].flatten()


  # Get the embedding vectors for each position using the trained embedding layer
  # We need to call another .detach().numpy() here since we were still getting the autograd error from PyTorch
  # We also needed to reshape the embeddings to remove an extra dimension, without the reshape(-1,84), their shapes were (995, 1, 84) now they're (995,84)

  combined_features_df = pd.DataFrame({
      **embedded_columns, # '**' unpacks a dictionary and passes the dict as key-value pais
      # this effectively replaces typing the below:
      #'rad_pos1_embed_1 : [..],
      #'rad_pos1_embed_2 : [..]

      'rad_pos1_last_gold': df['rad_pos1_predicted_last_gold'],
      'rad_pos2_last_gold': df['rad_pos2_predicted_last_gold'],
      'rad_pos3_last_gold': df['rad_pos3_predicted_last_gold'],
      'rad_pos4_last_gold': df['rad_pos4_predicted_last_gold'],
      'rad_pos5_last_gold': df['rad_pos5_predicted_last_gold'],
      'dire_pos1_last_gold': df['dire_pos1_predicted_last_gold'],
      'dire_pos2_last_gold': df['dire_pos2_predicted_last_gold'],
      'dire_pos3_last_gold': df['dire_pos3_predicted_last_gold'],
      'dire_pos4_last_gold': df['dire_pos4_predicted_last_gold'],
      'dire_pos5_last_gold': df['dire_pos5_predicted_last_gold'],
      'pos1_diff': df['pos1_diff'],
      'pos2_diff': df['pos2_diff'],
      'pos3_diff': df['pos3_diff'],
      'pos4_diff': df['pos4_diff'],
      'pos5_diff': df['pos5_diff'],
      'team_diff': df['team_diff'],
      'winner': df['winner']
  })

  return combined_features_df

In [ ]:
def predict_on_df_rf(df, lookback, horizon):

  # Add a new column for the predicted values to go in for each role and each team
  for i in range(1, 6):
      df[f'rad_pos{i}_predicted_last_gold'] = np.nan
      df[f'dire_pos{i}_predicted_last_gold'] = np.nan


  for index, row in df.iterrows():
    # Extract the necessary information from the row
    match_id = row['match_id']
    #print("Index: ", index, "Match ID: ", match_id)


    # Iterate and pull the time series for each role to store in dictionaries
    rad_pos_ts = {}  # Dictionary to store the timeseries
    rad_pos_heroes = {}
    dire_pos_ts = {}
    dire_pos_heroes = {}

    for iteration in range(1, 6):
        var_rad_name = f'rad_pos{iteration}_ts'
        var_rad_hero_name = f'rad_pos{iteration}_hero_id'
        var_dire_name = f'dire_pos{iteration}_ts'
        var_dire_hero_name = f'dire_pos{iteration}_hero_id'

        rad_pos_ts[var_rad_name] = np.array(json.loads(row[f'rad_pos{iteration}_gold'])).astype('float32')
        rad_pos_heroes[var_rad_hero_name] = row[f'rad_pos{iteration}']
        dire_pos_ts[var_dire_name] = np.array(json.loads(row[f'dire_pos{iteration}_gold'])).astype('float32')
        dire_pos_heroes[var_dire_hero_name] = row[f'dire_pos{iteration}']


    # Make copies of the time_series dictionaries to be scaled
    rad_pos_ts_scaled = rad_pos_ts.copy()
    dire_pos_ts_scaled = dire_pos_ts.copy()

    # Scale the time series
    for key, value in rad_pos_ts_scaled.items():
      scaled_ts = ConstantMinMaxScaler(value, min_gold, max_gold)
      rad_pos_ts_scaled[key] = scaled_ts

    for key, value in dire_pos_ts_scaled.items():
      scaled_ts = ConstantMinMaxScaler(value, min_gold, max_gold)
      dire_pos_ts_scaled[key] = scaled_ts

    # For each time_series in the dict, we need to create a window acceptable to the LSTM model
    for (scaled_key, scaled_data), (hero_key, hero_data) in zip(rad_pos_ts_scaled.items(), rad_pos_heroes.items()):
        # Access the key-value pairs from both dictionaries simultaneously
        #print(f"Scaled Key: {scaled_key}, Scaled Value: {scaled_data}")
        #print(f"Hero Key: {hero_key}, Hero Value: {hero_data}")
        #print("---")

        scaled_ts = scaled_data
        hero_id = hero_data

        X, y = create_windows(scaled_ts, lookback, horizon)
        # the LSTMs for use will be the 10 1 8 and the 10 1 1 (no embed)
        # if we pass the 10, 1 parameters to the create windows on a time series with 15, 20, 25..
        # we would want the 14th to 4th window.

        # len(timeseries) - lookback - horizon + 1 = 15 - 10 - 1 + 1 = 5
        # so for df_rf_15 there would be 5 tensors in X and 5 tensors in y
        # We only want to predict on the last tensor in X, X's shape should be 5 windows with 10 values

        #X = X[-1,:] # select the last row to predict on - causes error in LSTM Forward Method because dims on this and process embed arent same
        y = y[-1,:] # select the last row of y as a target(even though this won't be used)
        unscaled_y = ConstantUnScaler(y, min_gold = min_gold, max_gold = max_gold)

        #print("Shape of X: ", X.shape)
        #print("Shape of y: ", y.shape)
        # Create a batch with the selected time series
        batch = ((hero_id,), (X,), (y,))

        #Load our model that s declared like a global
        model.load_state_dict(model_dict)

        # Set the model to evaluation mode
        model.eval()

        # Disable gradient computation
        with torch.no_grad():
            # Forward pass
            outputs = model(batch)

            # Get the predicted values
            predicted = outputs.squeeze().numpy()
            #print("Rad Predicted: ", predicted)

        #print("Y: ", y)
        #print("Y[:-5]: ", y[:,-5:])
        #print("Predicted: ", predicted)
        #print("Predicted Shape: ", predicted.shape)
        # Unscaling values
        unscaled_predicted = ConstantUnScaler(predicted, min_gold=min_gold, max_gold=max_gold)
        unscaled_pred = unscaled_predicted[-1]
        #print("Unscaled Pred")

        # We need to assign the unscaled_pred to its appropriate column, to do this we check the position by
        # looking at the 7th character in the string that is scaled_key
        # print("scaled_key: ", scaled_key)
        # print("7th element of scaled_key: ", scaled_key[7])
        position = int(scaled_key[7])

        # Set our cell to the predicted value
        df.at[index, f'rad_pos{position}_predicted_last_gold'] = unscaled_pred

        ### ---- Dire Section

    for (scaled_key, scaled_data), (hero_key, hero_data) in zip(dire_pos_ts_scaled.items(), dire_pos_heroes.items()):
      scaled_ts = scaled_data
      hero_id = hero_data

      X, y = create_windows(scaled_ts, lookback, horizon)
      y = y[-1, :]
      unscaled_y = ConstantUnScaler(y, min_gold = min_gold, max_gold = max_gold)


      batch = ((hero_id,), (X,), (y,))

      model.load_state_dict(model_dict)
      model.eval()

      with torch.no_grad():
        outputs = model(batch)
        predicted = outputs.squeeze().numpy()

      unscaled_predicted = ConstantUnScaler(predicted, min_gold=min_gold, max_gold=max_gold)
      unscaled_pred = unscaled_predicted[-1]

      position = int(scaled_key[8])  # Extract the position number from the key
      df.at[index, f'dire_pos{position}_predicted_last_gold'] = unscaled_pred

  return df


In [ ]:
lookback = 10
horizon = 1

os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/lstm")
model_dict_10_1_rmse_8 = torch.load('10step_1horizon_RMSE_8.pth')
model_dict = model_dict_10_1_rmse_8

process_embedding = ProcessEmbedding(df2_allhero, embedding_dim=8, lookback=lookback) # we create the embedding vector on unsplit data to ensure all unique hero id's are contained
RMSE_embedding = ProcessEmbedding(df2_allhero, embedding_dim=8, lookback=lookback)
RMSE_embed_weights = model_dict_10_1_rmse_8['process_embedding.process_embeddings.weight']
RMSE_embedding.process_embeddings.weight = nn.Parameter(RMSE_embed_weights) # Load the old weights into the new embedding layer



input_size = process_embedding.embedding_dim + 1 #84 + 1
hidden_size = 64
num_layers = 2
output_size = 1  # Assuming you want to predict 5 values

model = LSTMModel(input_size, hidden_size, num_layers, output_size, process_embedding)

In [ ]:
#df_rf_15_predicted = predict_on_df_rf(df_rf_15, lookback = lookback, horizon = horizon)

In [ ]:
# checking to see if values are reasonable
#df_rf_15_predicted[['rad_pos1_last_gold','rad_pos2_predicted_last_gold', 'dire_pos1_last_gold','dire_pos1_predicted_last_gold']]

In [ ]:
#df_rf_15_predicted_dropped = drop_columns_for_predicted_df(df_rf_15_predicted)

In [ ]:
# checking to see if columns dropped correctly
#df_rf_15_predicted_dropped.columns

In [ ]:
#df_rf_15_predicted_dropped.columns

Index(['match_id', 'rad_pos1', 'rad_pos2', 'rad_pos3', 'rad_pos4', 'rad_pos5',
       'dire_pos1', 'dire_pos2', 'dire_pos3', 'dire_pos4', 'dire_pos5',
       'pos1_diff', 'pos2_diff', 'pos3_diff', 'pos4_diff', 'pos5_diff',
       'team_diff', 'minute', 'winner', 'rad_pos1_predicted_last_gold',
       'dire_pos1_predicted_last_gold', 'rad_pos2_predicted_last_gold',
       'dire_pos2_predicted_last_gold', 'rad_pos3_predicted_last_gold',
       'dire_pos3_predicted_last_gold', 'rad_pos4_predicted_last_gold',
       'dire_pos4_predicted_last_gold', 'rad_pos5_predicted_last_gold',
       'dire_pos5_predicted_last_gold'],
      dtype='object')

RF Test Data Indexing

Pulling the same indexes that were used as test data when evaluating the RF

In [ ]:
#dataframes = [df_rf_15, df_rf_20, df_rf_25, df_rf_30, df_rf_35, df_rf_40, df_rf_45]
iterations = [15, 20, 25, 30, 35, 40, 45]

for iteration in iterations:

    exec(f'df_rf_{iteration}_predicted = predict_on_df_rf(df_rf_{iteration}, lookback = lookback, horizon = horizon)')
    os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data/")
    exec(f"df_rf_{iteration}_predicted.to_csv('df_rf_{iteration}_predicted.csv')")
    exec(f'df_rf_{iteration}_predicted_dropped = drop_columns_for_predicted_df(df_rf_{iteration}_predicted)')
    exec(f"df_rf_{iteration}_predicted_dropped.to_csv('df_rf_{iteration}_predicted_dropped.csv')")


    os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data/test_data")
    y_test_index = np.load(f'rf_y_test_index_{iteration}.npy')

    exec(f'df_rf_{iteration}_predicted_dropped_test = df_rf_{iteration}_predicted_dropped.loc[y_test_index]')


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### RandomForest

- Instantiate the model
- Load the model
- Predict on the df_rf_iterations

In [ ]:
#df_rf_15_predicted_dropped_test.drop(columns = ['winner'])
#df_rf_15_predicted_dropped_test.columns



In [ ]:
dataframes = [df_rf_15, df_rf_20, df_rf_25, df_rf_30, df_rf_35, df_rf_40, df_rf_45]
iterations = [15, 20, 25, 30, 35, 40, 45]

for iteration in iterations:


  #Replace Hero_IDs with embeddings
  exec(f"df_rf_{iteration}_predicted_dropped_test_embed = get_hero_embedding_and_concat_features(df_rf_{iteration}_predicted_dropped_test)")

  # Load the chosen model
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/embed_rf")
  exec(f"loaded_model = joblib.load('rf_classifier_{iteration}.pkl')")
  exec(f"y = df_rf_{iteration}_predicted_dropped_test_embed['winner']")
  exec(f"X = df_rf_{iteration}_predicted_dropped_test_embed.drop(columns = 'winner')")
  exec(f"rf_predictions_{iteration} = loaded_model.predict(X)")

  # Create accuracy metrics
  exec(f"acc_{iteration} = accuracy_score(y,rf_predictions_{iteration})")
  exec(f"prec_{iteration} = precision_score(y,rf_predictions_{iteration}, average='weighted')")
  exec(f"recall_{iteration} = recall_score(y,rf_predictions_{iteration}, average='weighted')")
  exec(f"f1_score_{iteration} = f1_score(y,rf_predictions_{iteration}, average='weighted')")

  # Save Results
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/results")
  exec(f"rf_pred_metrics_{iteration} = pd.DataFrame([[acc_{iteration},prec_{iteration},recall_{iteration},f1_score_{iteration}]], columns =['Acc','Prec','Recall','F1'] )")
  exec(f"rf_pred_metrics_{iteration}.to_csv('RF_10lookback_1horizon_8embed_{iteration}_metrics.csv')")
  exec(f"print('RF_{iteration}:',rf_pred_metrics_{iteration})")

RF_15:        Acc     Prec   Recall       F1
0  0.70903  0.70938  0.70903  0.70856
RF_20:         Acc      Prec    Recall        F1
0  0.743945  0.744476  0.743945  0.743889
RF_25:         Acc      Prec    Recall        F1
0  0.779528  0.779515  0.779528  0.779473
RF_30:        Acc      Prec   Recall        F1
0  0.79602  0.796133  0.79602  0.795899
RF_35:         Acc      Prec    Recall        F1
0  0.798561  0.798347  0.798561  0.798288
RF_40:         Acc      Prec    Recall        F1
0  0.776471  0.798115  0.776471  0.773724
RF_45:         Acc   Prec    Recall        F1
0  0.644444  0.652  0.644444  0.646964


Random Forest No Embed

In [ ]:
lookback = 10
horizon = 1

os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/lstm_no_embed")
model_dict_10_1_rmse_1 = torch.load('10step_1horizon_RMSE_1.pth')
model_dict = model_dict_10_1_rmse_1

os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/")
%run LSTM_Model_Classes_no_embed.ipynb

input_size =  1 #84 + 1
hidden_size = 64
num_layers = 2
output_size = 1  # Assuming you want to predict 1 values - should match with horizon

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Rename Column Dict since the no embed model was expecting different column names

rename_dict = {
    'dire_pos1_predicted_last_gold': 'dire_pos1_last_gold',
    'dire_pos2_predicted_last_gold': 'dire_pos2_last_gold',
    'dire_pos3_predicted_last_gold': 'dire_pos3_last_gold',
    'dire_pos4_predicted_last_gold': 'dire_pos4_last_gold',
    'dire_pos5_predicted_last_gold': 'dire_pos5_last_gold',

    'rad_pos1_predicted_last_gold': 'rad_pos1_last_gold',
    'rad_pos2_predicted_last_gold': 'rad_pos2_last_gold',
    'rad_pos3_predicted_last_gold': 'rad_pos3_last_gold',
    'rad_pos4_predicted_last_gold': 'rad_pos4_last_gold',
    'rad_pos5_predicted_last_gold': 'rad_pos5_last_gold',
}

desired_order = ['rad_pos1_last_gold', 'rad_pos2_last_gold', 'rad_pos3_last_gold',
                 'rad_pos4_last_gold', 'rad_pos5_last_gold', 'dire_pos1_last_gold',
                 'dire_pos2_last_gold', 'dire_pos3_last_gold', 'dire_pos4_last_gold',
                 'dire_pos5_last_gold', 'pos1_diff', 'pos2_diff', 'pos3_diff',
                 'pos4_diff', 'pos5_diff', 'team_diff','winner']


In [ ]:
# feature_names = loaded_model.feature_names_in_
# feature_names

In [ ]:
#dataframes = [df_rf_15, df_rf_20, df_rf_25, df_rf_30, df_rf_35, df_rf_40, df_rf_45]
iterations = [15, 20, 25, 30, 35, 40, 45]

for iteration in iterations:

    exec(f'df_rf_{iteration}_no_embed_predicted = predict_on_df_rf(df_rf_{iteration}, lookback = lookback, horizon = horizon)')
    exec(f'df_rf_{iteration}_no_embed_predicted_dropped = drop_columns_for_no_embed(df_rf_{iteration}_no_embed_predicted)')
    exec(f"df_rf_{iteration}_no_embed_predicted_dropped_renamed = df_rf_{iteration}_no_embed_predicted_dropped.rename(columns = rename_dict)")
    exec(f"df_rf_{iteration}_no_embed_predicted_dropped_renamed_ordered =  df_rf_{iteration}_no_embed_predicted_dropped_renamed.reindex(columns=desired_order)")
    os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data/")
    exec(f"df_rf_{iteration}_no_embed_predicted_dropped_renamed_ordered.to_csv('df_rf_{iteration}_no_embed_predicted_dropped_renamed_ordered.csv')")

    os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data/test_data")
    y_test_index = np.load(f'rf_y_test_index_{iteration}.npy')

    exec(f'df_rf_{iteration}_no_embed_test = df_rf_{iteration}_no_embed_predicted_dropped_renamed_ordered.loc[y_test_index]')

In [ ]:
dataframes = [df_rf_15, df_rf_20, df_rf_25, df_rf_30, df_rf_35, df_rf_40, df_rf_45]
iterations = [15, 20, 25, 30, 35, 40, 45]

for iteration in iterations:


  #Replace Hero_IDs with embeddings
  #exec(f"df_rf_{iteration}_predicted_dropped_test_embed = get_hero_embedding_and_concat_features(df_rf_{iteration}_predicted_dropped_test)")

  # Load the chosen model
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/no_embed_rf")
  exec(f"loaded_model = joblib.load('rf_classifier_{iteration}.pkl')")
  exec(f"y = df_rf_{iteration}_no_embed_test['winner']")
  exec(f"X = df_rf_{iteration}_no_embed_test.drop(columns = 'winner')")
  exec(f"rf_predictions_no_embed_{iteration} = loaded_model.predict(X)")

  # Create accuracy metrics
  exec(f"acc_{iteration} = accuracy_score(y,rf_predictions_no_embed_{iteration})")
  exec(f"prec_{iteration} = precision_score(y,rf_predictions_no_embed_{iteration}, average='weighted')")
  exec(f"recall_{iteration} = recall_score(y,rf_predictions_no_embed_{iteration}, average='weighted')")
  exec(f"f1_score_{iteration} = f1_score(y,rf_predictions_no_embed_{iteration}, average='weighted')")

  # Save Results
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/results")
  exec(f"rf_pred_no_embed_metrics_{iteration} = pd.DataFrame([[acc_{iteration},prec_{iteration},recall_{iteration},f1_score_{iteration}]], columns =['Acc','Prec','Recall','F1'] )")
  exec(f"rf_pred_no_embed_metrics_{iteration}.to_csv('RF_10lookback_1horizon_no_embed_{iteration}_metrics.csv')")
  exec(f"print('RF_{iteration}:',rf_pred_no_embed_metrics_{iteration})")

RF_15:         Acc      Prec    Recall        F1
0  0.735786  0.737046  0.735786  0.735014
RF_20:         Acc      Prec    Recall        F1
0  0.771626  0.771711  0.771626  0.771632
RF_25:         Acc      Prec    Recall        F1
0  0.771654  0.771753  0.771654  0.771512
RF_30:         Acc      Prec    Recall        F1
0  0.781095  0.781324  0.781095  0.780899
RF_35:         Acc      Prec    Recall        F1
0  0.776978  0.777291  0.776978  0.777094
RF_40:         Acc      Prec    Recall        F1
0  0.788235  0.800133  0.788235  0.787059
RF_45:         Acc   Prec    Recall        F1
0  0.688889  0.696  0.688889  0.691093


### XGBoost Predictions

In [ ]:
param_grid = {
    'n_estimators': [200, 300],
    'max_depth':  [10],
    'learning_rate': [0.01 ,0.1],
    'subsample': [0.8 ,1.0],
    'colsample_bytree': [0.8 , 1.0],
    'gamma': [0, 0.1, 0.3],
    'min_child_weight': [1, 5, 10]
}

In [ ]:
dataframes = [df_rf_15, df_rf_20, df_rf_25, df_rf_30, df_rf_35, df_rf_40, df_rf_45]
iterations = [15, 20, 25, 30, 35, 40, 45]

for iteration in  iterations:

    # Ensuring we only test on the test indices
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data/test_data")
  exec(f"xg_y_test_index_{iteration} = np.load(f'xg_y_test_index_{iteration}.npy')")
  exec(f'xg_df_{iteration}_predicted_dropped_test = df_rf_{iteration}_predicted_dropped.loc[xg_y_test_index_{iteration}]')

      #Replace Hero_IDs with embeddings
  exec(f"xg_df_{iteration}_predicted_dropped_test_embed = get_hero_embedding_and_concat_features(xg_df_{iteration}_predicted_dropped_test)")

  exec(f"xg_{iteration}_y = xg_df_{iteration}_predicted_dropped_test_embed['winner']")
  exec(f"xg_{iteration}_X = xg_df_{iteration}_predicted_dropped_test_embed.drop(columns = 'winner')")

  # Swap to binary results
  exec(f"y_{iteration}_binary = xg_{iteration}_y.replace({{'dire': 0, 'radiant': 1}})")

    # Load the chosen model
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/embed_xgb")
  exec(f"loaded_model = joblib.load('XGB_{iteration}_model_embed.pkl')")

  # Make Predictions
  exec(f"XGB_predictions_{iteration} = loaded_model.predict(xg_{iteration}_X)")

  # Create accuracy metrics
  exec(f"acc_{iteration} = accuracy_score(y_{iteration}_binary,XGB_predictions_{iteration})")
  exec(f"prec_{iteration} = precision_score(y_{iteration}_binary,XGB_predictions_{iteration}, average='weighted')")
  exec(f"recall_{iteration} = recall_score(y_{iteration}_binary,XGB_predictions_{iteration}, average='weighted')")
  exec(f"f1_score_{iteration} = f1_score(y_{iteration}_binary,XGB_predictions_{iteration}, average='weighted')")

  # Save Results
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/results")
  exec(f"xgb_pred_metrics_{iteration} = pd.DataFrame([[acc_{iteration},prec_{iteration},recall_{iteration},f1_score_{iteration}]], columns =['Acc','Prec','Recall','F1'] )")
  exec(f"xgb_pred_metrics_{iteration}.to_csv('XGB_10lookback_1horizon_8embed_{iteration}_metrics.csv')")
  exec(f"print('XGB_{iteration}:',xgb_pred_metrics_{iteration})")


XGB_15:         Acc      Prec    Recall        F1
0  0.712375  0.713211  0.712375  0.711638
XGB_20:         Acc      Prec    Recall        F1
0  0.754325  0.754488  0.754325  0.754325
XGB_25:         Acc      Prec    Recall        F1
0  0.783465  0.784047  0.783465  0.783158
XGB_30:         Acc      Prec    Recall       F1
0  0.771144  0.771348  0.771144  0.77094
XGB_35:         Acc     Prec    Recall        F1
0  0.769784  0.76948  0.769784  0.769472
XGB_40:         Acc     Prec    Recall        F1
0  0.752941  0.76138  0.752941  0.751982
XGB_45:         Acc      Prec    Recall        F1
0  0.711111  0.734387  0.711111  0.713987


XGBoost No Embed

In [ ]:
dataframes = [df_rf_15, df_rf_20, df_rf_25, df_rf_30, df_rf_35, df_rf_40, df_rf_45]
iterations = [15, 20, 25, 30, 35, 40, 45]

for iteration in  iterations:


  exec(f'df_rf_{iteration}_no_embed_test = df_rf_{iteration}_no_embed_predicted_dropped_renamed_ordered.loc[y_test_index]')
  exec(f"y = df_rf_{iteration}_no_embed_test['winner']")
  exec(f"X = df_rf_{iteration}_no_embed_test.drop(columns = ['winner'])")

  # Swap to binary results
  exec(f"y_{iteration}_binary = y.replace({{'dire': 0, 'radiant': 1}})")

    # Load the chosen model
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/no_embed_xgb")
  exec(f"loaded_model = joblib.load('XGB_no_embed_{iteration}_model.pkl')")

  # Make Predictions
  exec(f"XGB_noemb_predictions_{iteration} = loaded_model.predict(X)")

  # Create accuracy metrics
  exec(f"acc_{iteration} = accuracy_score(y_{iteration}_binary,XGB_noemb_predictions_{iteration})")
  exec(f"prec_{iteration} = precision_score(y_{iteration}_binary,XGB_noemb_predictions_{iteration}, average='weighted')")
  exec(f"recall_{iteration} = recall_score(y_{iteration}_binary,XGB_noemb_predictions_{iteration}, average='weighted')")
  exec(f"f1_score_{iteration} = f1_score(y_{iteration}_binary,XGB_noemb_predictions_{iteration}, average='weighted')")

  # Save Results
  os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models/results")
  exec(f"xgb_noemb_pred_metrics_{iteration} = pd.DataFrame([[acc_{iteration},prec_{iteration},recall_{iteration},f1_score_{iteration}]], columns =['Acc','Prec','Recall','F1'] )")
  exec(f"xgb_noemb_pred_metrics_{iteration}.to_csv('XGB_10lookback_1horizon_no_embed_{iteration}_metrics.csv')")
  exec(f"print('XGB_{iteration}:',xgb_noemb_pred_metrics_{iteration})")


XGB_15:         Acc      Prec    Recall        F1
0  0.777778  0.784975  0.777778  0.779835
XGB_20:         Acc      Prec    Recall        F1
0  0.866667  0.866667  0.866667  0.866667
XGB_25:    Acc      Prec  Recall        F1
0  0.8  0.809486     0.8  0.801195
XGB_30:    Acc      Prec  Recall        F1
0  0.8  0.798618     0.8  0.795407
XGB_35:         Acc      Prec    Recall     F1
0  0.822222  0.838735  0.822222  0.824
XGB_40:         Acc      Prec    Recall        F1
0  0.888889  0.909465  0.888889  0.887783
XGB_45:         Acc     Prec    Recall       F1
0  0.711111  0.71417  0.711111  0.71229
